<a href="https://colab.research.google.com/github/flatlalejo1982/Phase-4-Project-Execution/blob/main/Phase_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Phase 4 Project Execution**

In [1]:
#Load core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

**1.Data Preparation**

In [2]:
#Load Data

data = pd.read_csv("airdata.csv", sep = ",")
data.head()

,date,Station code,Address,Latitude,Longitude,SO2,NO2,CO,O3,PM2.5,PM10
0,1/1/2017 0:00,101,"Kuril Bishow Road, Dhaka, Bangladesh",23.820612,90.421011,0.04,0.059,1.2,0.0525,57,73
1,1/1/2017 1:00,101,"Kuril Bishow Road, Dhaka, Bangladesh",23.820612,90.421011,0.04,0.058,1.2,0.0525,59,71
2,1/1/2017 2:00,101,"Kuril Bishow Road, Dhaka, Bangladesh",23.820612,90.421011,0.04,0.056,1.2,0.0525,59,70
3,1/1/2017 3:00,101,"Kuril Bishow Road, Dhaka, Bangladesh",23.820612,90.421011,0.04,0.056,1.2,0.0525,58,70
4,1/1/2017 4:00,101,"Kuril Bishow Road, Dhaka, Bangladesh",23.820612,90.421011,0.03,0.051,1.2,0.0525,61,69


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155405 entries, 0 to 155404
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   date          155405 non-null  object 
 1   Station code  155405 non-null  int64  
 2   Address       155405 non-null  object 
 3   Latitude      155405 non-null  float64
 4   Longitude     155405 non-null  float64
 5   SO2           155405 non-null  float64
 6   NO2           155405 non-null  float64
 7   CO            155405 non-null  float64
 8   O3            155405 non-null  float64
 9   PM2.5         155405 non-null  int64  
 10  PM10          155405 non-null  int64  
dtypes: float64(6), int64(3), object(2)
memory usage: 13.0+ MB


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Calculation of the Air Quality Index (AQI) for various pollutants**




**Define the EPA ranges (in µg/m³ or ppm depending on the contaminant)**

In [6]:
#Create EPA range table
rangos = {
    "SO2": pd.DataFrame({
        'Bmin': [0, 36, 76, 186, 305, 605],
        'Bmax': [35, 75, 185, 304, 604, 1004],
        'Imin': [0, 51, 101, 151, 201, 301],
        'Imax': [50, 100, 150, 200, 300, 500],
        'Categoria': ['Good', 'Moderate', 'Harmful/sensitive Groups', 'Poor', 'Very Poor', 'Dangerous']
    }),
    "NO2": pd.DataFrame({
        'Bmin': [0, 54, 101, 361, 650, 1250],
        'Bmax': [53, 100, 360, 649, 1249, 2049],
        'Imin': [0, 51, 101, 151, 201, 301],
        'Imax': [50, 100, 150, 200, 300, 500],
        'Categoria': ['Good', 'Moderate', 'Harmful/sensitive Groups', 'Poor', 'Very Poor', 'Dangerous']
    }),
    "CO": pd.DataFrame({
        'Bmin': [0.0, 4.5, 9.5, 12.5, 15.5, 30.5],
        'Bmax': [4.4, 9.4, 12.4, 15.4, 30.4, 50.4],
        'Imin': [0, 51, 101, 151, 201, 301],
        'Imax': [50, 100, 150, 200, 300, 500],
        'Categoria': ['Good', 'Moderate', 'Harmful/sensitive Groups', 'Poor', 'Very Poor', 'Dangerous']
    }),
    "O3": pd.DataFrame({
        'Bmin': [0.000, 0.055, 0.071, 0.086, 0.106, 0.201],
        'Bmax': [0.054, 0.070, 0.085, 0.105, 0.200, 0.604],
        'Imin': [0, 51, 101, 151, 201, 301],
        'Imax': [50, 100, 150, 200, 300, 500],
        'Categoria': ['Good', 'Moderate', 'Harmful/sensitive Groups', 'Poor', 'Very Poor', 'Dangerous']
    }),
    "PM2.5": pd.DataFrame({
        'Bmin': [0.0, 12.1, 35.5, 55.5, 150.5, 250.5],
        'Bmax': [12.0, 35.4, 55.4, 150.4, 250.4, 500.4],
        'Imin': [0, 51, 101, 151, 201, 301],
        'Imax': [50, 100, 150, 200, 300, 500],
        'Categoria': ['Good', 'Moderate', 'Harmful/sensitive Groups', 'Poor', 'Very Poor', 'Dangerous']
    }),
    "PM10": pd.DataFrame({
        'Bmin': [0, 55, 155, 255, 355, 425],
        'Bmax': [54, 154, 254, 354, 424, 604],
        'Imin': [0, 51, 101, 151, 201, 301],
        'Imax': [50, 100, 150, 200, 300, 500],
        'Categoria': ['Good', 'Moderate', 'Harmful/sensitive Groups', 'Poor', 'Very Poor', 'Dangerous']
    }),
}
